In [31]:
import pandas as pd
from langchain.chains import LLMChain, SequentialChain


In [32]:
import prompts as p
from langchain.prompts import PromptTemplate


DATA_PATH = '../../datasets/downsampled_50_50.csv'

SAVED_PROMPTS_PATH = '../../datasets/saved_prompts.csv'

# Load data
df = pd.read_csv(DATA_PATH)

df.fillna('', inplace=True)

prompt_name = 'prompt1_en_short_temp_0_gpt-4-1106-preview'
prompt = p.prompt1_en_short

MODEL = "gpt-4-1106-preview"

MAX_TOKENS = 15


In [33]:
# generate column with prompt_name if not exists
if 'prompt_name' not in df.columns:
    df[prompt_name] = ""

In [34]:
df.head(30)

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,cluster_id_left,id_right,brand_right,...,prompt_1_zero_shot_CoT_eval2_temp_0_gpt3.5_0301,match_prompt_autogpt4_fewshot_2_eval2_temp_0_gpt3.5-0301_v2,match_prompt_basic_zeroshot_2_eval2_temp_0_gpt3.5-0301,match_prompt_manual_fewshot_1_eval2_temp_0_gpt3.5-0301_v2,prompt1_en_short_temp_0_gpt3.5-0301_with_brand,pprompt1_instruct_temp_0_gpt-3.5-turbo-instruct,prompt1_en_short_instruct_temp_0_gpt-3.5-turbo-instruct_v2,match_prompt_basic_zeroshot_1_eval2_temp_0_gpt3.5-0301_v2,prompt1_en_short_temp_0_gpt-3.5-turbo-1106,prompt1_en_short_temp_0_gpt-4-1106-preview
0,0,24080197,,DJI Mavic Mini Drone Fly More Combo,"Mavic Mini, the everyday flycam. Open your sky...",649.00,CAD,1709252,92817369,Fujifilm,...,False,False,False,False,,False,False,False,False,
1,1,94788371,,Instax Mini Rainbow Film 10 Pack,Brighten the place up with this pack of rainbo...,14.99,GBP,1374090,10718715,,...,False,False,False,False,,False,False,False,False,
2,2,14016048,,Blackmagic Design ATEM Production Studio 4K Sw...,Blackmagic Design ATEM Production Studio 4K Sw...,1.695E3,USD,1263992,52586114,,...,False,False,False,False,,False,False,False,False,
3,3,62248973,Instax,Instax Fujifilm | Instax SQUARE Instant Film -...,,14.99,CAD,1118780,63595157,,...,False,False,False,False,,False,False,False,False,
4,4,25011125,,Hikvision DVR 16 CH fino a 8Mpx Turbo 3.0 DS-7...,Hikvision DVR 16 CH fino a 8Mpx Turbo 3.0 DS-7...,3.726E2,EUR,1336313,10718715,,...,False,False,False,False,,False,False,False,False,
5,5,60568221,,TAG Heuer Formula 1 Chronograph 43mm CAZ1010.F...,,1169.75,EUR,2107019,31469161,Sony,...,False,False,False,False,,False,False,False,False,
6,6,27700752,,SanDisk Extreme PRO SD UHS-II USB 3.0,,99,PLN,137095,86534825,SanDisk,...,False,False,False,False,,False,False,False,False,
7,7,64446755,Brother,"TN-200 sort lasertoner, original Brother TN200...",Ny original Brother sort tonerpatron til Broth...,339.00,DKK,5223239,65569829,,...,False,False,False,False,,False,False,False,False,
8,8,54979311,,TP-Link TL-WN781ND Netcard PCI-eX Wireless 150 mb,,,,466493,34393388,,...,False,False,False,False,,False,False,False,False,
9,9,89974024,,Refill #014: Kraft Paper Notebook,Cardboard Paper notebook refill for the Travel...,7.9,EUR,1857459,58146758,,...,False,False,False,False,,False,False,False,False,


In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI


In [36]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [37]:
from langfuse.callback import CallbackHandler
handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [38]:
def cut_length(string ,length = 250):
    return string[0:length]

In [39]:

# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none"))
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none"))



# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none")) + "\nBrand:" + str(df["brand_left"][1] or "none") 
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none")) + "\nBrand:" + str(df["brand_right"][1] or "none") 

In [40]:
saved_prompts_df = pd.read_csv(SAVED_PROMPTS_PATH)

def saveResponse(prompt, response, isMatch, index):
    # create new column if not exists 
    if "prompt_" + prompt_name not in saved_prompts_df.columns:
        saved_prompts_df["prompt_" + prompt_name] = ""
        saved_prompts_df["response_" + prompt_name] = ""
        saved_prompts_df["isMatch_" + prompt_name] = ""


    
    # save response prompt and isMatch to df at index
    saved_prompts_df.at[index, "prompt_" + prompt_name] = prompt
    saved_prompts_df.at[index, "response_" + prompt_name] = response
    saved_prompts_df.at[index, "isMatch_" + prompt_name] = isMatch
    
    

In [41]:
def evaluate(df, prompt, temp=0, start = 0):
    # start = len(df) - start - 1
    if (MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name=MODEL, temperature=temp,max_tokens=MAX_TOKENS)
        print("Using OpenAI")
    else:
        llm = ChatOpenAI(model_name=MODEL, temperature=temp,max_tokens=MAX_TOKENS)
    for i in range(start, len(df)):
        # i = len(df) - index - 1
        print(i)
        # Creating Prompot and sending request and printing informations
        brandl = ""
        brandr = ""
        # if (df["brand_left"][i]):
        #     brandl = "\nBrand: " + str(df["brand_left"][i])
        # if (df["brand_right"][i]):
        #     brandr = "\nBrand: " + str(df["brand_right"][i])
        
        product1 = "Title: " + cut_length(str(df["title_left"][i] or "none")) + brandl+ "\nDescription: " + cut_length(str(df["description_left"][i] or "none")) 
        product2 = "Title: " + cut_length(str(df["title_right"][i] or "none")) + brandr + "\nDescription: " + cut_length(str(df["description_right"][i] or "none")) 
        
        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        
        try:
            response = chain.run(p1=product1, p2=product2, callbacks=[handler])        
        except Exception as e:
            print(e)
            i -= 1
            continue

        formatted_prompt = chain.prompt.format_prompt(p1=product1, p2=product2).text
        if(i < 5):
            print(formatted_prompt)
            print("__\n__")
            print(response)
            print("__\n__")
        
        #Processing Response    
        isMatch = False
        if ("True" in response or "true" in response or "yes" in response):
            isMatch = True
        elif ("False" in response or "false" in response or "no" in response):
            isMatch = False
        
        df[prompt_name][i] = isMatch


        saveResponse(formatted_prompt, response, isMatch, i)

In [42]:
evaluate(df, prompt,temp=0, start=0)


0
Are the following two products the same?
Product 1: Title: DJI Mavic Mini Drone Fly More Combo
Description: Mavic Mini, the everyday flycam. Open your sky with the compact, yet powerful Mavic Mini. Weighing under 250 grams, together with the DJI Fly APP, you'll enjoy a simplified flying experience that is almost as light as the average smartphone.
Product 2: Title: Fujifilm Instax Mini Instant Film - Monochrome (10 Shots)
Description: Monochrome (Black & White) Instant Film for Fujifilm Mini Instant Cameras10 Shots per PackISO 800High Gloss FinishSuperb Grain QualityFast Image FormationGreater Resistance to Age-Induced ChangesSingle-plastic Composition for Easier Disposal

Answer with 'True' or 'False'.
__
__
False
__
__
1


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_3765/2523417414.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


Are the following two products the same?
Product 1: Title: Instax Mini Rainbow Film 10 Pack
Description: Brighten the place up with this pack of rainbow coloured film! Top tip: one of these would look fab in our Instax Mini Acrylic Photo Block!Includes pack of 10 shots
Product 2: Title: 1 Fujifilm Instax Square Film Black Frame
Description: none

Answer with 'True' or 'False'.
__
__
False
__
__
2


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_3765/2523417414.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


Are the following two products the same?
Product 1: Title: Blackmagic Design ATEM Production Studio 4K Switcher
Description: Blackmagic Design ATEM Production Studio 4K SwitchATEM Production Studio 4K lets you connect up to 8 SD, HD or Ultra HD 4K video cameras, disk recorders and computers to do true broadcast quality work. You get all the features you expect from a profe
Product 2: Title: Western Digital Blue PC 2.5\" 250 GB Serial ATA III
Description: 250 GB, 2.5\", SATA 6Gb/s, 540/500 MB/s

Answer with 'True' or 'False'.
__
__
False
__
__
3


...

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_3765/2523417414.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


219


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_3765/2523417414.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


220


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_3765/2523417414.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


In [43]:
# df


In [44]:
# saved_prompts_df

In [45]:
# write df to csv
df.to_csv(DATA_PATH, index=False)
saved_prompts_df.to_csv(SAVED_PROMPTS_PATH, index=False)
